In [ ]:
import pandas as pd
import torch
df = pd.read_csv("/content/data_banknote_authentication.txt",sep=",")
df

,0,1,2,3,4
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [ ]:
x_train = df[["0","1","2","3"]].values
y_train = df["4"].values

In [ ]:
x_train.shape

(1372, 4)

In [ ]:
import numpy as np
np.bincount(y_train)

array([762, 610])

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
  def __init__(self,x,y):
    self.features = torch.tensor(x,dtype=torch.float32)
    self.labels = torch.tensor(y,dtype=torch.float32)
  def __getitem__(self, index):
    x1 = self.features[index]
    y1 = self.labels[index]
    return x1,y1
  def __len__(self):
    return self.labels.shape[0]

In [ ]:
train_size = int(x_train.shape[0]*0.80)
train_size

1097

In [ ]:
val_size = int(x_train.shape[0]) - train_size
val_size

275

In [ ]:
dataset = MyDataset(x_train,y_train)
torch.manual_seed(1)
train_set,val_set = torch.utils.data.random_split(dataset,[train_size,val_size])
train_loader = DataLoader(
    dataset = train_set,
    batch_size = 10,
    shuffle = True,
)
val_loader = DataLoader(
    dataset = val_set,
    batch_size = 10,
    shuffle = True,
)

In [ ]:
class LogisticRegression(torch.nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.linear = torch.nn.Linear(num_features,1)
  def forward(self,x):
    logits = self.linear(x)
    probas = torch.sigmoid(logits)
    return probas

In [ ]:
import torch.nn.functional as F
torch.manual_seed(1)
model = LogisticRegression(num_features=4)
optimizer = torch.optim.SGD(model.parameters(),lr=0.07)
num_epochs = 65
for epoch in range(num_epochs):
  model = model.train()
  for batch_idx, (features, class_labels) in enumerate(train_loader):
    probas = model(features)
    loss = F.binary_cross_entropy(probas,class_labels.view(probas.shape))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if not batch_idx%20:
       print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
                   f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
                   f' | Loss: {loss:.2f}')

Epoch: 001/065 | Batch 000/110 | Loss: 1.30
Epoch: 001/065 | Batch 020/110 | Loss: 0.21
Epoch: 001/065 | Batch 040/110 | Loss: 0.32
Epoch: 001/065 | Batch 060/110 | Loss: 0.09
Epoch: 001/065 | Batch 080/110 | Loss: 0.05
Epoch: 001/065 | Batch 100/110 | Loss: 0.07
Epoch: 002/065 | Batch 000/110 | Loss: 0.16
Epoch: 002/065 | Batch 020/110 | Loss: 0.06
Epoch: 002/065 | Batch 040/110 | Loss: 0.18
Epoch: 002/065 | Batch 060/110 | Loss: 0.07
Epoch: 002/065 | Batch 080/110 | Loss: 0.08
Epoch: 002/065 | Batch 100/110 | Loss: 0.09
Epoch: 003/065 | Batch 000/110 | Loss: 0.04
Epoch: 003/065 | Batch 020/110 | Loss: 0.09
Epoch: 003/065 | Batch 040/110 | Loss: 0.05
Epoch: 003/065 | Batch 060/110 | Loss: 0.21
Epoch: 003/065 | Batch 080/110 | Loss: 0.06
Epoch: 003/065 | Batch 100/110 | Loss: 0.02
Epoch: 004/065 | Batch 000/110 | Loss: 0.04
Epoch: 004/065 | Batch 020/110 | Loss: 0.02
Epoch: 004/065 | Batch 040/110 | Loss: 0.11
Epoch: 004/065 | Batch 060/110 | Loss: 0.06
Epoch: 004/065 | Batch 080/110 |

In [ ]:
def compute_acc(model,dataloader):
  model =model.eval()
  correct = 0.0
  total_examples = 0
  for batch_idx, (features, class_labels) in enumerate(dataloader):
    with torch.no_grad():
      probas = model(features)

    pred = torch.where(probas>0.5,1,0)
    lab = class_labels.view(pred.shape).to(pred.dtype)

    compare = lab == pred
    correct += torch.sum(compare)
    total_examples += len(compare)
  return correct/total_examples

In [ ]:
train_acc = compute_acc(model,train_loader)
print(f"Accuracy: {train_acc*100:.2f}%")
val_acc = compute_acc(model,val_loader)
print(f"Accuracy: {val_acc*100:.2f}%")

Accuracy: 98.81%
Accuracy: 99.27%
